In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from datetime import timedelta
from datetime import date
import os 
import warnings
import time

#SYMBOL = "EURUSD"

def Login():
    account = 124363966
    password = "Kc80121@"
    server = "Exness-MT5Trial7"
    mt5.initialize()
    if not mt5.login(account, password=password, server=server):
        print("login() failed")
        mt5.shutdown()
    



def ENTRY(SYMBOL):
    def Get_Position(symbol):
        
        positions = mt5.positions_get(symbol=symbol)
        position_symbol= []
        order_symbol= []
        orders = mt5.orders_get(symbol=symbol)
    
    
        for position in positions :
    
            position_symbol.append(position.symbol)
    
        for order in orders:
    
            order_symbol.append(order.symbol)
    
        #print(position_symbol)
        #print(order_symbol)
        cur_pos = False
        for i in position_symbol:
            if i == symbol:
                cur_pos = True
    
        pen_pos = False
        for i in order_symbol:
    
            if i == symbol:
                pen_pos = True
    
    
        ANSWER = True
    
    
        if cur_pos == pen_pos == False:
            ANSWER = False
        elif cur_pos==pen_pos==True:
            ANSWER == True
        else: ANSWER = True
    
    
        #print(get_pos)
        return ANSWER

    if Get_Position(SYMBOL)==False:



        # Request historical data from MetaTrader 5
        symbol = SYMBOL
        timeframe_1min = mt5.TIMEFRAME_M1  # 1-minute timeframe for close price
        timeframe_5min = mt5.TIMEFRAME_M5  # 5-minute timeframe for SAR, VWAP, and Bollinger Bands
        today = date.today()
        yesterday = today - timedelta(days=1)

        from_date = pd.Timestamp(yesterday)
        to_date = pd.Timestamp(datetime.now())
        rates_1min = mt5.copy_rates_range(symbol, timeframe_1min, from_date, to_date)
        rates_5min = mt5.copy_rates_range(symbol, timeframe_5min, from_date, to_date)


        # vwap cal
        symbol = SYMBOL
        timeframe_1min_vwap = mt5.TIMEFRAME_M1  # 1-minute timeframe for close price
        from_date_vwap = pd.Timestamp("2024-04-25")
        to_date_vwap = pd.Timestamp(datetime.now())
        rates_vwap_1min = mt5.copy_rates_range(symbol, timeframe_1min_vwap, from_date_vwap, to_date_vwap)
        data_vwap_1min = pd.DataFrame(rates_vwap_1min)

        # # Convert historical data to DataFrame
        # data_1min = pd.DataFrame(rates_1min)
        # data_5min = pd.DataFrame(rates_5min)
        # af_init = 0.02
        # step_af = 0.02
        # end_af = 0.2
        #
        # data_5min['trend'] = 0
        # data_5min['sar'] = 0.0
        # data_5min['real_sar'] = 0.0
        # data_5min['ep'] = 0.0
        # data_5min['af'] = 0.0
        #
        # data_5min['trend'][1] = 1 if data_5min['close'][1] > data_5min['close'][0] else -1
        # data_5min['sar'][1] = data_5min['high'][0] if data_5min['trend'][1] > 0 else data_5min['low'][0]
        # data_5min.at[1, 'real_sar'] = data_5min['sar'][1]
        # data_5min['ep'][1] = data_5min['high'][1] if data_5min['trend'][1] > 0 else data_5min['low'][1]
        # data_5min['af'][1] = af_init
        # data_1min = pd.DataFrame(rates_1min)
        # data_5min = pd.DataFrame(rates_5min)
        # Convert historical data to DataFrame
        data_1min = pd.DataFrame(rates_1min)
        data_5min = pd.DataFrame(rates_5min)

        # Print the DataFrame and its columns for debugging
        # print("Contents of data_5min DataFrame:")
        # print(data_5min)
        warnings.filterwarnings('ignore')
        # Check if the DataFrame is not empty and contains necessary columns
        if not data_5min.empty and 'close' in data_5min.columns and 'high' in data_5min.columns and 'low' in data_5min.columns:
            af_init = 0.02
            step_af = 0.02
            end_af = 0.2

            data_5min['trend'] = 0
            data_5min['sar'] = 0.0
            data_5min['real_sar'] = 0.0
            data_5min['ep'] = 0.0
            data_5min['af'] = 0.0

            # Set initial values based on the first two data points
            data_5min['trend'][1] = 1 if data_5min['close'][1] > data_5min['close'][0] else -1
            data_5min['sar'][1] = data_5min['high'][0] if data_5min['trend'][1] > 0 else data_5min['low'][0]
            data_5min.at[1, 'real_sar'] = data_5min['sar'][1]
            data_5min['ep'][1] = data_5min['high'][1] if data_5min['trend'][1] > 0 else data_5min['low'][1]
            data_5min['af'][1] = af_init
        else:
            print("DataFrame 'data_5min' is empty or missing required columns."+ " " + SYMBOL)



        #data_5min.to_csv("SAR.csv")
        af_init = 0.02
        step_af = 0.02
        end_af = 0.2

        data_5min['trend'] = 0
        data_5min['sar'] = 0.0
        data_5min['real_sar'] = 0.0
        data_5min['ep'] = 0.0
        data_5min['af'] = 0.0

        data_5min['trend'][1] = 1 if data_5min['close'][1] > data_5min['close'][0] else -1
        data_5min['sar'][1] = data_5min['high'][0] if data_5min['trend'][1] > 0 else data_5min['low'][0]
        data_5min.at[1, 'real_sar'] = data_5min['sar'][1]
        data_5min['ep'][1] = data_5min['high'][1] if data_5min['trend'][1] > 0 else data_5min['low'][1]
        data_5min['af'][1] = af_init

        for i in range(2, len(data_5min)):
            temp = data_5min['sar'][i - 1] + data_5min['af'][i - 1] * (data_5min['ep'][i - 1] - data_5min['sar'][i - 1])
            if data_5min['trend'][i - 1] < 0:
                data_5min.at[i, 'sar'] = max(temp, data_5min['high'][i - 1], data_5min['high'][i - 2])
                temp = 1 if data_5min['sar'][i] < data_5min['high'][i] else data_5min['trend'][i - 1] - 1
            else:
                data_5min.at[i, 'sar'] = min(temp, data_5min['low'][i - 1], data_5min['low'][i - 2])
                temp = -1 if data_5min['sar'][i] > data_5min['low'][i] else data_5min['trend'][i - 1] + 1
            data_5min.at[i, 'trend'] = temp
            if data_5min['trend'][i] < 0:
                temp = min(data_5min['low'][i], data_5min['ep'][i - 1]) if data_5min['trend'][i] != -1 else \
                data_5min['low'][i]
            else:
                temp = max(data_5min['high'][i], data_5min['ep'][i - 1]) if data_5min['trend'][i] != 1 else \
                data_5min['high'][i]
            data_5min.at[i, 'ep'] = temp
            if np.abs(data_5min['trend'][i]) == 1:
                temp = data_5min['ep'][i - 1]
                data_5min.at[i, 'af'] = af_init
            else:
                temp = data_5min['sar'][i]
                if data_5min['ep'][i] == data_5min['ep'][i - 1]:
                    data_5min.at[i, 'af'] = data_5min['af'][i - 1]
                else:
                    data_5min.at[i, 'af'] = min(end_af, data_5min['af'][i - 1] + step_af)
            data_5min.at[i, 'real_sar'] = temp

        # Remove SAR at (0,0)
        data_5min = data_5min[data_5min['sar'] != 0]
        #print(data_5min['close'][1])

        # VWAP Calculation
        data_vwap_1min['tp'] = (data_vwap_1min['high'] + data_vwap_1min['low'] + data_vwap_1min['close']) / 3
        data_vwap_1min['tpv'] = data_vwap_1min['tp'] * data_vwap_1min['tick_volume']
        data_vwap_1min['cum_volume'] = data_vwap_1min['tick_volume'].cumsum()
        data_vwap_1min['cum_tpv'] = data_vwap_1min['tpv'].cumsum()
        data_vwap_1min['vwap'] = data_vwap_1min['cum_tpv'] / data_vwap_1min['cum_volume']

        ema_period = 1440
        data_1min['ema'] = data_1min['close'].ewm(span=ema_period, adjust=False).mean()

        # Bollinger Bands Calculation
        bb_period = 20
        data_5min['bb_mid'] = data_5min['close'].rolling(window=bb_period).mean()
        data_5min['bb_std'] = data_5min['close'].rolling(window=bb_period).std()
        data_5min['bb_upper'] = data_5min['bb_mid'] + 2 * data_5min['bb_std']
        data_5min['bb_lower'] = data_5min['bb_mid'] - 2 * data_5min['bb_std']

        # Add bullish and bearish columns to the data_5min DataFrame
        data_5min['bullish'] = data_5min['sar'] < data_5min['close']
        data_5min['bearish'] = data_5min['sar'] > data_5min['close']

        # CALCULATIONS
        last_sar_value = data_5min['sar'].iloc[-1]
        ema_prev = data_1min['ema'].iloc[-1]
        vwap_prev = data_vwap_1min['vwap'].iloc[-1]
        bollinger_prev = data_5min['bb_mid'].iloc[-1]

        previous_close = data_1min['close'].shift(1)
        close_prev = previous_close.iloc[-1]

        def is_bearish_signal(close_price, vwap, ema, bb_lower):
            if close_price < vwap and close_price < ema and close_price < bb_lower:
                return True
            else:
                return False

        def is_bullish_signal(close_price, vwap, ema, bb_lower):
            if close_price > vwap and close_price > ema and close_price > bb_lower:
                return True
            else:
                return False

        warnings.filterwarnings('ignore')
        Trend_Bear = is_bearish_signal(close_prev, vwap_prev, ema_prev, bollinger_prev)

        Trend_Bull = is_bullish_signal(close_prev, vwap_prev, ema_prev, bollinger_prev)

        Sar_0 = data_5min['sar'].iloc[0]
        Sar_1 = data_5min['sar'].iloc[-1]
        Sar_2 = data_5min['sar'].iloc[-2]
        Sar_3 = data_5min['sar'].iloc[-3]
        Sar_4 = data_5min['sar'].iloc[-4]

        close_5 = previous_close.iloc[-6]

        # Get the last 15 candle high values
        last_15_candle_highs = data_1min['high'].tail(30)
        last_15_candle_lows = data_1min['low'].tail(30)

        # Find the maximum among the last 10 candle highs
        max_high = last_15_candle_highs.max()
        max_low = last_15_candle_lows.min()
        Sar_1 = data_5min['sar'].iloc[-1]
        Sar_2 = data_5min['sar'].iloc[-2]
        Sar_3 = data_5min['sar'].iloc[-3]
        Sar_4 = data_5min['sar'].iloc[-4]

        def fibonacci_levels(low_price, high_price):
            # Calculate the price range
            price_range = high_price - low_price

            # Define Fibonacci ratios
            fibonacci_ratios = [-0.5, 0.25, 0.5, 1.0]

            # Calculate Fibonacci levels
            fibonacci_levels = {}
            for ratio in fibonacci_ratios:
                level_price = high_price + ratio * price_range
                fibonacci_levels[ratio] = level_price

            return fibonacci_levels

        # Example usage
        def Spread(symbol):
            
            # Get tick information for the symbol
            tick = mt5.symbol_info_tick(symbol)

            if tick is not None:
                # Calculate spread
                spread = tick.ask - tick.bid
                print(f"Spread for {symbol}: {spread} points")
                
            else:
                spread = 0.0
                print(f"Failed to get tick information for {symbol}")

            return spread

        def TP_SL(sar, HL):
            #sar = low_price
            #HL = high_price
            fib_levels = fibonacci_levels(sar, HL)
            # print("Fibonacci levels between", low_price, "and", high_price, ":")
            Entry_price = fib_levels[-0.5]
            StopLoss = sar
            TakeProfit = fib_levels[0.5]
            return Entry_price, StopLoss, TakeProfit



        def SAR_PREV():
            #print(data_5min['close'][1])
            #data_1min.to_csv("Sar1m.csv")
            Sar_0 = data_5min['sar'].iloc[0]
            Sar_1 = data_5min['sar'].iloc[-1]
            Sar_2 = data_5min['sar'].iloc[-2]
            Sar_3 = data_5min['sar'].iloc[-3]
            Sar_4 = data_5min['sar'].iloc[-4]
            Sar_5 = data_5min['sar'].iloc[-5]
            # BUY
            def BUY():
                LEVELS=TP_SL(max_low,max_high)
                Entry_Price = LEVELS[0]
                Take_Profit = LEVELS[2]
                #print(spread)
                spread = Spread(SYMBOL)
                #print(spread)
                Stop_Loss = LEVELS[1]-spread
                symbol = SYMBOL
                #trade_type = mt5.ORDER_TYPE_BUY_LIMIT  # Trade type: Buy Limit
                volume = 0.1  # Volume in lots
                price = Entry_Price  # Price at which you want to place the order
                deviation = 0  # Deviation from the current price in points
                stop_loss = Stop_Loss  # Stop-loss level
                take_profit = Take_Profit # Take-profit level
                comment = "Buy Limit Order with SL and TP"  # Comment for the order

                # Define the order request
                request = {
                    "action": mt5.TRADE_ACTION_PENDING,
                    "symbol": symbol,
                    "volume": volume,
                    "type": mt5.ORDER_TYPE_BUY_LIMIT,
                    "price": price,
                    "sl" : stop_loss,
                    "tp" : take_profit,
                    "deviation": 0,
                    "type_time": mt5.ORDER_TIME_GTC,
                    "Type_filling": mt5.ORDER_FILLING_IOC,
                }

                result = mt5.order_send(request)

                # Check if the order was sent successfully
                if result is not None:
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print("Buy Limit order with SL and TP placed successfully! Order:", result.order)
                    else:
                        print("Order failed! Error code:", result.retcode)
                else:
                    print("Order failed! Result is None."+ " " + SYMBOL)

                # Shutdown MetaTrader 5 connection


            def SELL():
                LEVELS = TP_SL(max_high, max_low)
                Entry_Price = LEVELS[0]
                Take_Profit = LEVELS[2]
                spread = Spread(SYMBOL)
                Stop_Loss = LEVELS[1]+spread
                symbol = SYMBOL
                #trade_type = mt5.ORDER_TYPE_SELL_LIMIT  # Trade type: Suy Limit
                volume = 0.1  # Volume in lots
                price = Entry_Price  # Price at which you want to place the order
                deviation = 0  # Deviation from the current price in points
                stop_loss = Stop_Loss  # Stop-loss level
                take_profit = Take_Profit  # Take-profit level
                comment = "Sell Limit Order with SL and TP"  # Comment for the order

                # Define the order request
                request = {
                    "action": mt5.TRADE_ACTION_PENDING,
                    "symbol": symbol,
                    "volume": volume,
                    "type": mt5.ORDER_TYPE_SELL_LIMIT,
                    "price": price,
                    "sl": stop_loss,
                    "tp": take_profit,
                    "deviation": 0,
                    "type_time": mt5.ORDER_TIME_GTC,
                    "Type_filling": mt5.ORDER_FILLING_IOC,
                }

                # Send the order
                result = mt5.order_send(request)

                # Check if the order was sent successfully
                if result is not None:
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print("Sell Limit order with SL and TP placed successfully! Order:", result.order)
                    else:
                        print("Order failed! Error code:", result.retcode)
                else:
                    print("Order failed! Result is None."+ " " + SYMBOL)

                # Shutdown MetaTrader 5 connection

            if Trend_Bull==True and Trend_Bear == False:

                print("BULL"+ " " + SYMBOL)
                if Sar_1 < close_prev and Sar_0 < close_prev:


                     if Sar_0 > Sar_1 or Sar_1 > Sar_2 or Sar_2 > Sar_3 :
                         if Sar_3 < Sar_4:
                         #data_5min.to_csv("SAR.csv")
                            print("Buying"+ " " + SYMBOL)
                            #print(Sar_1)
                            print(TP_SL(max_low, max_high))
                            #
                            BUY()

                         else: print("Waiting for Down Sars,  BULLISH MARKET"+ " " + SYMBOL)

                     else:

                        print("Wait buy"+ " " + SYMBOL)


            elif Trend_Bear==True and Trend_Bull==False:
                 print("BEAR"+ " " + SYMBOL)
                 if Sar_1 > close_prev and Sar_0 > close_prev:

                    if Sar_0 < Sar_1 or Sar_1 < Sar_2 or Sar_2 < Sar_3 :

                        if Sar_3 > Sar_4:
                            print("Selling"+ " " + SYMBOL)
                            print(TP_SL(max_high,max_low))
                            SELL()
                            #BUY()
                        else: print("Waiting for UP Sars ,  BEARISH MARKET")
                    else:
                        print("Wait sell " + " " + SYMBOL )


            else:
                print("Wait"+ " " + SYMBOL)


        SAR_PREV()
    else: 
        symbol = SYMBOL
        timeframe_1min = mt5.TIMEFRAME_M1
        today = date.today()
        yesterday = today - timedelta(days=1)
        
        from_date = pd.Timestamp(yesterday)
        to_date = pd.Timestamp(datetime.now())
        rates_1min = mt5.copy_rates_range(symbol, timeframe_1min, from_date, to_date)
        data_1min = pd.DataFrame(rates_1min)
        
        orders = mt5.orders_get(symbol=SYMBOL)
        current_close = data_1min['close'].shift(1)
        prev_close_pen = current_close.iloc[-1]
        def cancel_order (symbol):
            pending_order = mt5.orders_get(symbol=symbol)
            #print(pending_order)
            order1 = pending_order[0]
            request = {
                'action' : mt5.TRADE_ACTION_REMOVE,
                'order': order1.ticket
            }
            mt5.order_send(request)
        
        
        empty_tuple = ()
        if orders is not empty_tuple:
            for order in orders:
                tp_c = order.tp
                sl_c = order.sl
                tick = order.ticket
        
            if tp_c > sl_c:
                if prev_close_pen > tp_c:
                    cancel_order(symbol)
                    print("Buy Order Close TP" + " " + SYMBOL)
                elif prev_close_pen < sl_c:
                    cancel_order(symbol)
                    print("Buy Order Close SL" + " " + SYMBOL)
                else:
                    print("Order ALREADY RUNNING" + " " + SYMBOL)
            elif tp_c < sl_c:
                if prev_close_pen < tp_c:
                    cancel_order(symbol)
                    print("Sell Order Close TP" + " " + SYMBOL)
                elif prev_close_pen > sl_c:
                    cancel_order(symbol)
                    print("Sell Order Close SL " + " " + SYMBOL)
                else:
                    print("Order ALREADY RUNNING" + " " + SYMBOL)
        else: 
            print("Position ALREADY RUNNING" + " " + SYMBOL)

In [2]:
Login()

In [ ]:
import os
from IPython.display import clear_output

for i in range(10000000):
    pairs=['EURUSD', 'EURJPY', 'EURGBP', 'EURCAD', 'EURNZD', 'EURCHF', 'AUDCAD', 'AUDCHF', 'AUDJPY', 'AUDNZD', 'AUDUSD', 'CADCHF', 'CADJPY', 'CHFJPY', 'GBPAUD', 'GBPJPY', 'NZDCAD', 'NZDJPY', 'NZDCHF', 'USDCAD', 'USDCHF', 'USDJPY', 'XAUUSD','BTCUSD']
    #pairs = ['BTCUSD', 'ETHUSD','SOLUSD','BNBUSD','ADAUSD','BNBUSD','BCHUSD','CAKEUSD','DODGEUSD','1INCHUSD','MATICUSD','XRPUSD' ]
    for i in pairs:
        print(" ")
        ENTRY(i)
        print(" ")
        
    clear_output(wait=True)    
    time.sleep(45)
    
mt5.shutdown()

 
Wait EURUSD
 
 
Position ALREADY RUNNING EURJPY
 
 
Wait EURGBP
 
 
Wait EURCAD
 
 
Wait EURNZD
 
 
Wait EURCHF
 
 
BULL AUDCAD
 
 
